# What is this?
Its a Self Driving Car Using Convolutional Neural Networks Via Using NVIDIA Model Architecture for End To End Self Driving Car And For Simulation Using Udacity Open Source Unity Self Driving Car Simulator.

I have created my own dataset using Udacity simulator self-driving car training mode and after that, **I used a bigger dataset for other sources** on GitHub and trained my model on that dataset. For convenience you can use my data https://lnkd.in/dqwDUi7 to train Your model For more good datasets u can check these out

https://lnkd.in/dsXVYXR
Format: Image, latitude, longitude, gear, brake, throttle, steering angles and speed

Credits- 
NVIDIA's paper: End to End Learning for Self-Driving Cars for the inspiration and model structure.

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import matplotlib.image as mpimg

In [ ]:
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Convolution2D,Dense,MaxPooling2D,Dropout,Flatten
import cv2
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import pandas as pd
import random
import ntpath

In [ ]:
datadir = '/kaggle/input/selfdriving-car-dataset/data-master/'

In [ ]:
# columnas para entrenar
columns = ['center','left','right','steering','throttle','reverse','speed']

In [ ]:
# leer datos
dataset = pd.read_csv(os.path.join(datadir,'/kaggle/input/selfdriving-car-dataset/data-master/driving_log.csv'),names=columns)

In [ ]:
os.path.join(datadir,'/kaggle/input/selfdriving-car-dataset/data-master/driving_log.csv')

In [ ]:
dataset.head()

In [ ]:
def removePath(path):
  base,tail = ntpath.split(path)
  return tail

In [ ]:
dataset['center'] = dataset['center'].apply(removePath)

In [ ]:
dataset['left'] = dataset['left'].apply(removePath)

In [ ]:
dataset['right'] = dataset['right'].apply(removePath)

In [ ]:
dataset.head()

In [ ]:
num_bins = 25

In [ ]:
hist,bins = np.histogram(dataset['steering'],num_bins)

In [ ]:
print(hist)
print(bins)

In [ ]:
center = (bins[:-1]+bins[1:])*0.5

In [ ]:
center

In [ ]:
center1 = []
for i in range(0,len(bins)-1):
  x = (bins[i] + bins[i+1]) * 0.5
  center1.append(x)

In [ ]:
center1

In [ ]:
# graficar angulos de manejo

threshold = 500
plt.figure(figsize=(15,10))
plt.bar(center,hist,width=0.05)
plt.xticks(np.linspace(-1,1,25),rotation=90)
(x1,x2) = (np.min(dataset['steering']),np.max(dataset['steering']))
(y1,y2) = (threshold,threshold)
plt.title('Steering Angles')
plt.plot((x1,x2),(y1,y2))

In [ ]:
remove_list = []
for i in range(num_bins):
  List = []
  for j in range(len(dataset['steering'])):
    if dataset['steering'][j] >= bins[i] and dataset['steering'][j] <= bins[i+1]:
      List.append(j)
  List = shuffle(List)
  List = List[threshold:]
  remove_list.extend(List)

In [ ]:
len(dataset['steering']) 

In [ ]:
len(remove_list)

In [ ]:
dataset.drop(dataset.index[remove_list],inplace=True)

In [ ]:
hist,_ = np.histogram(dataset['steering'],num_bins)

In [ ]:
plt.bar(center,hist,width=0.05)
plt.xticks(np.linspace(-1,1,25),rotation=90)
(x1,x2) = (np.min(dataset['steering']),np.max(dataset['steering']))
(y1,y2) = (threshold,threshold)
plt.title('Steering Angles')
plt.plot((x1,x2),(y1,y2))

In [ ]:
dataset.iloc[1]

In [ ]:
datadir

In [ ]:
def loadImageSteering(datadir,dataset):
  imagePath = []
  steeringPath = []
  for i in range(len(dataset)):
    center = dataset.iloc[i][0]
    steering = float(dataset.iloc[i][3])
    imagePath.append(os.path.join(datadir,center))
    steeringPath.append(steering)
  imagePath = np.asarray(imagePath)
  steeringPath = np.asarray(steeringPath)
  return imagePath,steeringPath

In [ ]:
dataset.iloc[0][0]

In [ ]:
imagePath,steeringPath = loadImageSteering(datadir+'/IMG',dataset)

In [ ]:
imagePath[0]

In [ ]:
len(steeringPath)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(imagePath,steeringPath,random_state=6,test_size=0.2)

In [ ]:
len(x_train)

In [ ]:
plt.figure(figsize=(15,10))
plt.hist(y_train,bins=num_bins,width=0.05)
plt.xticks(np.linspace(-1,1,25),rotation=45)
plt.title("Training Dataset")
plt.show()

In [ ]:
plt.figure(figsize=(15,10))
plt.hist(y_test,bins=num_bins,width=0.05)
plt.xticks(np.linspace(-1,1,25),rotation=45)
plt.title("Testing Dataset")
plt.show()

In [ ]:
def imagePreprocessing(img):
  img = mpimg.imread(img)
  img = img[60:135,:,:]
  img = cv2.cvtColor(img,cv2.COLOR_RGB2YUV)
  img = cv2.GaussianBlur(img,(3,3),0)
  img = cv2.resize(img,(200,66))
  img = img/255
  return img

In [ ]:
image = imagePath[1]
image = mpimg.imread(image)
fig,axs = plt.subplots(1,2,figsize=(15,10))
fig.tight_layout()
axs[0].imshow(image)
axs[0].grid(False)
axs[0].set_title("Original Image")
axs[1].imshow(imagePreprocessing(imagePath[1]))
axs[1].grid(False)
axs[1].set_title("Precessed Image")
plt.show()

In [ ]:
x_train = np.array(list(map(imagePreprocessing,x_train)))

In [ ]:
x_test = np.array(list(map(imagePreprocessing,x_test)))

In [ ]:
# from google.colab import files

In [ ]:
#upload = files.upload()

In [ ]:
# img = plt.imread('nvidia architectue.png')

In [ ]:
# plt.figure(figsize=(15,50))
# plt.imshow(img)
# plt.title("NVIDIA Architecture")

In [ ]:
def nvidiaModel():
  model = Sequential()
  model.add(Convolution2D(24,(5,5),strides=(2,2),input_shape=(66,200,3),activation="elu"))
  model.add(Convolution2D(36,(5,5),strides=(2,2),activation="elu"))
  model.add(Convolution2D(48,(5,5),strides=(2,2),activation="elu")) 
  model.add(Convolution2D(64,(3,3),activation="elu"))   
  model.add(Convolution2D(64,(3,3),activation="elu"))
  model.add(Dropout(0.5))
  
  model.add(Flatten())
  
  model.add(Dense(100,activation="elu"))
  model.add(Dropout(0.5))
  
  model.add(Dense(50,activation="elu"))
  model.add(Dropout(0.5))
  
  model.add(Dense(10,activation="elu"))
  model.add(Dropout(0.5))
  
  model.add(Dense(1))
  model.compile(optimizer=Adam(lr=1e-3),loss="mse")
  
  return model

In [ ]:
model = nvidiaModel()

In [ ]:
model.summary()

In [ ]:
h = model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=30,batch_size=100,shuffle=1,verbose=1)

In [ ]:
plt.plot(h.history['loss'])
plt.plot(h.history['val_loss'])

In [ ]:
model.save('car.h5')

In [ ]:
# from google.colab import files

In [ ]:
# files.download('car.h5')

In [ ]:
type('car.h5')